In [69]:
#imports
import pulp as p
from pulp import value
import pandas as pd
import numpy as np
import matplotlib as plt

In [47]:
# Pickle-Datei laden
df_2019 = pd.read_pickle('daten.pkl')
df = df_2019

anschaffungskosten_speicher = 6000
anschaffunskosten_pumpe = 7800
jahresrechnung_heizkosten_vor_pumpe = 2370

T_innen = 20  # Ziel-Innentemperatur in °C
Wohnfläche = 150  # Wohnfläche in m²
Wärmebedarf_pro_m2 = 60  # Wärmebedarf in W/m² bei normaler Dämmung
T_norm_aussen = -12  # Normaußentemperatur (zur Normierung des Temperaturunterschieds) in °C
heizungstemperatur=35 #Fußbodenheizung optimalerweise
wirkungsgrad=0.8
cop_vorfaktor=0.2
Delta_T_max = T_innen - T_norm_aussen

# Maximaler Temperaturunterschied
Delta_T_max = T_innen - T_norm_aussen

# Heizlastberechnung für jede Stunde
#heizlast = [(Wärmebedarf_pro_m2 * Wohnfläche * (T_innen - T_außen) / Delta_T_max) for T_außen in temperatur]

# Stundenweise Heizlast in kW umrechnen (von W zu kW)
#heizlast_kW = [round(last / 1000, 2) for last in heizlast]

# Berechnung der COP-Werte
#t_warm = 273.15 + heizungstemperatur
#t_kalt = [x + 273.15 for x in temperatur]
#cop_werte = [cop_vorfaktor * (t_warm / (t_warm - kalt)) for kalt in t_kalt]

# Kosten pro kW(h) Wärme berechnen, in diesem Fall kWh, da sich alle Werte auf Stunden beziehen
#kosten_pro_kW = [strompreis / cop for cop, strompreis in zip(cop_werte, strompreise)]

# Muss aus COP/Strompeis gibt mir wieviel Euro es zu Zeit t kostet um einen kW Wärme zu erzeugen
#c = kosten_pro_kW


In [48]:
#Hier Grafik und Herleitung für: Von Wetter und Wärmebedarf zu COP, von COP und Strompreis zu C  

In [49]:
#Hier grafische Erklärung für die Nebenbedingungen

In [73]:
# Anzahl der Tage die betrachet werden sollen
weeks = 20
T = int(len(df['Strompreis'])/365*7*weeks)

# Maximale Werte (Beispiele, ersetze sie durch deine tatsächlichen Werte)
p_max = 10
s_max = 50
s_laden = 5
s_entladen = 5
P_bat = 5
E_bat = 20
eta_bat = 0.9
COP = 3
c_pur = 0.2  # EUR/kWh
c_feedin = 0.1  # EUR/kWh
dt = 1  # Stunde
B_start = 20  # Anfangsfüllstand des Wärmespeichers
l_start = 10  # Anfangsfüllstand der Batterie

In [74]:
# Beispielhafte Parameterdefinitionen

# Außentemperaturen (°C) für jede Stunde
T_außen = df_2019['Geschätzte Temperatur'].values

Strompreise = df_2019['Strompreis'].values

# Heizlastfaktor und beheizte Fläche
alpha = 50  # W/m²°C
T_innen = 20  # °C
Flaeche = 100  # m²

# Berechnung der Wärmenachfrage
d = [alpha * (T_innen - T_außen[t]) * Flaeche / 1000 for t in range(T)]  # in kW

# Elektrische Nachfrage des Haushalts ohne Wärmepumpe (kW) für jede Stunde
D_Haushalt = df_2019['demand'].values

# Berechnung der PV-Erzeugung (kW)
PV = df_2019['pv'].values

# Kosten pro kW Wärme (EUR/kW) für jede Stunde
c_t = [1] * T  # Konstant 1 EUR/kW in jeder Stunde

## Zielfunktion
- Minimierung der Gesamtkosten:
  - Wärmepumpenkosten $c_t \cdot P_t$
  - Strombezugskosten $y^{pur}_t \cdot c^{pur} \cdot dt$
  - Abzüglich Einspeisevergütung $y^{pv,feedin}_t \cdot c^{feedin} \cdot dt$

$$
\min \sum_{t=1}^{T} \left( c_t \cdot P_t + y^{pur}_t \cdot c^{pur} \cdot dt - y^{pv,feedin}_t \cdot c^{feedin} \cdot dt \right)
$$

## Nebenbedingungen

### 1. Wärmepumpe und Wärmespeicher
- **Grenzen der Wärmeerzeugung:**

$$
0 \leq P_t \leq P_{\text{max}}, \quad \forall t \in \{1, \dots, T\}
$$

- **Grenzen für Speicherein- und -ausspeicherung:**

$$
0 \leq x^{\text{in}}_t \leq X_{\text{in,max}}, \quad 0 \leq x^{\text{out}}_t \leq X_{\text{out,max}}, \quad \forall t \in \{1, \dots, T\}
$$

- **Grenzen für den Wärmespeicherfüllstand:**

$$
0 \leq B_t \leq B_{\text{max}}, \quad \forall t \in \{1, \dots, T\}
$$

- **Wärmebilanz:**

$$
P_t + x^{\text{out}}_t = d_t + x^{\text{in}}_t, \quad \forall t \in \{1, \dots, T\}
$$

- **Dynamik des Speicherfüllstands:**

$$
B_{t+1} = B_t + x^{\text{in}}_t - x^{\text{out}}_t, \quad \forall t \in \{1, \dots, T-1\}
$$

$$
B_1 = B_{\text{start}} + x^{\text{in}}_1 - x^{\text{out}}_1
$$

### 2. Stromsystem (Netz, PV, Batteriespeicher)
- **Deckung der elektrischen Nachfrage:**

$$
y^{pur}_t + y^{pv,self}_t + y^{s,dc}_t - y^{s,ch}_t + y^{heat,bat}_t + y^{heat,pv}_t = D_t, \quad \forall t \in \{1, \dots, T\}
$$

- **Aufteilung der PV-Erzeugung:**

$$
y^{pv,self}_t + y^{pv,feedin}_t = PV_t, \quad \forall t \in \{1, \dots, T\}
$$

- **Grenzen für Batterie-Lade- und Entladeleistung:**

$$
y^{s,dc}_t \leq P^{bat}, \quad y^{s,ch}_t \leq P^{bat}, \quad \forall t \in \{1, \dots, T\}
$$

- **Dynamik des Batterie-Füllstands:**

$$
l^s_{t+1} = l^s_t + y^{s,ch}_t \cdot \eta_{bat} \cdot dt - y^{s,dc}_t \cdot dt, \quad \forall t \in \{1, \dots, T-1\}
$$

- **Grenzen des Batterie-Füllstands:**

$$
0 \leq l^s_t \leq E^{bat}, \quad \forall t \in \{1, \dots, T\}
$$

- **Elektrischer Betrieb der Wärmepumpe:**

$$
D_t = D_t^{\text{Haushalt}} + \frac{P_t}{\mathrm{COP}}, \quad \forall t \in \{1, \dots, T\}
$$

- **Wärmepumpe kann Strom aus der Batterie beziehen:**

$$
y^{heat,bat}_t \leq y^{s,dc}_t, \quad \forall t \in \{1, \dots, T\}
$$

- **Wärmepumpe kann Strom direkt von der Photovoltaik-Anlage beziehen:**

$$
y^{heat,pv}_t \leq y^{pv,self}_t, \quad \forall t \in \{1, \dots, T\}
$$

## Zusammenfassung der Entscheidungsvariablen
- Wärmepumpenkosten $c_t \cdot P_t$
- Strombezug aus dem Netz $y^{pur}_t$
- Einspeisung ins Netz $y^{pv,feedin}_t$
- PV-Eigennutzung $y^{pv,self}_t$
- Entladung der Batterie $y^{s,dc}_t$
- Ladung der Batterie $y^{s,ch}_t$
- Batteriebezug der Wärmepumpe $y^{heat,bat}_t$
- PV-Bezug der Wärmepumpe $y^{heat,pv}_t$
- Speicherfüllstand Wärmespeicher $B_t$
- Batterie-Füllstand $l^s_t$
- Speicherein $x^{\text{in}}_t$
- Speicheraus $x^{\text{out}}_t$

## Parameter
- Kosten pro kW Wärme $c_t$
- Konstanter Strombezugspreis $c^{pur}$ (EUR/kWh)
- Einspeisevergütung $c^{feedin}$ (EUR/kWh)
- Zeitschrittlänge $dt$ (z.B. 1 Stunde)
- Maximale Leistung der Wärmepumpe $P_{\text{max}}$
- Maximale Einspeicherleistung $X^{\text{in,max}}$
- Maximale Ausspeicherleistung $X^{\text{out,max}}$
- Maximale Speicherkapazität Wärmespeicher $B_{\text{max}}$
- Anfangsfüllstand Wärmespeicher $B_{\text{start}}$
- PV-Erzeugung $PV_t$ in Stunde $t$
- Elektrische Nachfrage $D_t$ in Stunde $t$ (inklusive Wärmepumpe)
- Batteriekapazität $E^{bat}$
- Maximale Lade-/Entladeleistung der Batterie $P^{bat}$
- Batteriewirkungsgrad $\eta_{bat}$
- Leistungszahl der Wärmepumpe $\mathrm{COP}$

---


In [75]:
#initialize model and its direction (memory space to assign objective function and constraints)
model = p.LpProblem("problem", p.LpMinimize)

In [76]:
# Erzeugung von Variablen (Bereits hier werden die Nebenbedingungen für Unter- und Obergrenzen hinzugefügt)

# Aus unserem Modell

# Erzeugt die Wärmepumpenvariable und fügt Grenzen hinzu
P = [p.LpVariable(f"p_{t}", lowBound=0, upBound=p_max) for t in range(T)]

# Erzeugt die Variable für den thermischen Speicherfüllstand und fügt Grenzen hinzu
B = [p.LpVariable(f"B_{t}", lowBound=0, upBound=s_max) for t in range(T)]

# Erzeugt die Ladevariable und fügt Grenzen hinzu
x_in = [p.LpVariable(f"x_in_{t}", lowBound=0, upBound=s_laden) for t in range(T)]

# Erzeugt die Entladevariable und fügt Grenzen hinzu
x_out = [p.LpVariable(f"x_out_{t}", lowBound=0, upBound=s_entladen) for t in range(T)]

# Aus Basismodell

# Erzeugt Variable für den Strombezug aus dem Netz und fügt untere Grenze hinzu
y_pur = [p.LpVariable(f"y_pur_{t}", lowBound=0) for t in range(T)]

# Erzeugt Variable für die PV-Eigennutzungsvariable und fügt untere Grenze hinzu
y_pv_self = [p.LpVariable(f"y_pv_self_{t}", lowBound=0) for t in range(T)]

# Erzeugt Variable für die PV-Einspeisungsvariable und fügt untere Grenze hinzu
y_pv_feedin = [p.LpVariable(f"y_pv_feedin_{t}", lowBound=0) for t in range(T)]

# Erzeugt Variable für die Batterieladung und fügt Grenzen hinzu
y_s_ch = [p.LpVariable(f"y_s_ch_{t}", lowBound=0, upBound=P_bat) for t in range(T)]

# Erzeugt Variable für die Batterientladung und fügt Grenzen hinzu
y_s_dc = [p.LpVariable(f"y_s_dc_{t}", lowBound=0, upBound=P_bat) for t in range(T)]

# Erzeugt Variable für den Batteriefüllstand und fügt Grenzen hinzu
l_s = [p.LpVariable(f"l_s_{t}", lowBound=0, upBound=E_bat) for t in range(T)]

# Neu aus Kombination beider Modelle

# Erzeugt Variable für den Wärmepumpenstrombezug durch Batterie und fügt untere Grenze hinzu
y_heat_bat = [p.LpVariable(f"y_heat_bat_{t}", lowBound=0) for t in range(T)]

# Erzeugt Variable für den Wärmepumpenstrombezug durch PV-Anlage und fügt untere Grenze hinzu
y_heat_pv = [p.LpVariable(f"y_heat_pv_{t}", lowBound=0) for t in range(T)]

# Erzeugt die Gesamtstromnachfragevariable und fügt eine untere Grenze hinzu
D = [p.LpVariable(f"D_{t}", lowBound=0) for t in range(T)]


In [77]:
# Zielfunktion: Minimierung der Gesamtkosten
# Kosten der Wärmepumpe + Kosten für Strombezug - Einnahmen durch PV-Einspeisung

cost_pump = p.lpSum([c_t[t] * P[t] for t in range(T)])
cost_grid = p.lpSum([y_pur[t] * c_pur * dt for t in range(T)])
revenue_pv = p.lpSum([y_pv_feedin[t] * c_feedin * dt for t in range(T)])

objective = p.lpSum([cost_pump, cost_grid, -revenue_pv])

In [78]:
#Komplexere Nebenbedingungen (nicht nur obere und untere Begrenzungen)

# Liste für aller sonstigen Nebenbedingungen
constraints = []

# **1. Wärmepumpe und Wärmespeicher**

for t in range(T):
    # Grenzen der Wärmeerzeugung
    constraints.append(P[t] <= p_max)

    # Grenzen für Speicherein- und -ausspeicherung
    constraints.append(x_in[t] <= s_laden)
    constraints.append(x_out[t] <= s_entladen)

    # Wärmebilanz
    # P_t + x_out_t = d_t + x_in_t
    constraints.append(P[t] + x_out[t] == d[t] + x_in[t])

# Dynamik des Speicherfüllstands
for t in range(T-1):
    # B_{t+1} = B_t + x_in_t - x_out_t
    constraints.append(B[t+1] == B[t] + x_in[t] - x_out[t])

# Anfangsfüllstand des Wärmespeichers
constraints.append(B[0] == B_start + x_in[0] - x_out[0])

# **2. Stromsystem (Netz, PV, Batteriespeicher)**

for t in range(T):
    # Deckung der elektrischen Nachfrage
    # y_pur_t + y_pv_self_t + y_s_dc_t - y_s_ch_t + y_heat_bat_t + y_heat_pv_t = D_t
    constraints.append(y_pur[t] + y_pv_self[t] + y_s_dc[t] - y_s_ch[t] + y_heat_bat[t] + y_heat_pv[t] == D[t])

    # Aufteilung der PV-Erzeugung
    # y_pv_self_t + y_pv_feedin_t = PV_t
    constraints.append(y_pv_self[t] + y_pv_feedin[t] == PV[t])

    # Wärmepumpe darf nur so viel Strom aus der Batterie beziehen, wie entladen wird
    # y_heat_bat_t <= y_s_dc_t
    constraints.append(y_heat_bat[t] <= y_s_dc[t])

    # Wärmepumpe darf nur so viel Strom direkt von der PV beziehen, wie verfügbar ist
    # y_heat_pv_t <= y_pv_self_t
    constraints.append(y_heat_pv[t] <= y_pv_self[t])

# Dynamik des Batterie-Füllstands
for t in range(T-1):
    # l_s_{t+1} = l_s_t + y_s_ch_t * eta_bat * dt - y_s_dc_t * dt
    constraints.append(l_s[t+1] == l_s[t] + y_s_ch[t] * eta_bat * dt - y_s_dc[t] * dt)

# Anfangsfüllstand der Batterie
constraints.append(l_s[0] == l_start)

# Grenzen des Batterie-Füllstands sind bereits durch die Variable-Definition gesetzt

# Elektrischer Betrieb der Wärmepumpe
for t in range(T):
    # D_t = D_Haushalt_t + P_t / COP
    constraints.append(D[t] == D_Haushalt[t] + P[t] * (1.0 / COP))



In [79]:
# Erstelle das Optimierungsproblem
model = p.LpProblem("Energieoptimierung", p.LpMinimize)

# Füge die Zielfunktion hinzu
model += objective

# Füge alle Nebenbedingungen hinzu
for constraint in constraints:
    model += constraint

# Löse das Problem
model.solve()

# Ergebnisse anzeigen
print("Status:", p.LpStatus[model.status])
print("Minimale Kosten:", p.value(model.objective))
for t in range(T):
    print(f"Wärmeleistung P_{t}: {P[t].varValue}")
    print(f"Speicherfüllstand B_{t}: {B[t].varValue}")
    print(f"Ladeleistung x_in_{t}: {x_in[t].varValue}")
    print(f"Entladeleistung x_out_{t}: {x_out[t].varValue}")
    print(f"Strombezug aus dem Netz y_pur_{t}: {y_pur[t].varValue}")
    print(f"PV-Eigennutzung y_pv_self_{t}: {y_pv_self[t].varValue}")
    print(f"PV-Einspeisung y_pv_feedin_{t}: {y_pv_feedin[t].varValue}")
    print(f"Batterieladung y_s_ch_{t}: {y_s_ch[t].varValue}")
    print(f"Batterientladung y_s_dc_{t}: {y_s_dc[t].varValue}")
    print(f"Batteriebezug der Wärmepumpe y_heat_bat_{t}: {y_heat_bat[t].varValue}")
    print(f"PV-Bezug der Wärmepumpe y_heat_pv_{t}: {y_heat_pv[t].varValue}")
    print(f"Batteriefüllstand l_s_{t}: {l_s[t].varValue}")
    print("-" * 30)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/juliandomnik/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m4/qcxwdmxd5b94sx7ctbs447dc0000gn/T/cd75f24da73844389587dcadf6b4ba6b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/m4/qcxwdmxd5b94sx7ctbs447dc0000gn/T/cd75f24da73844389587dcadf6b4ba6b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 36965 COLUMNS
At line 144482 RHS
At line 181443 BOUNDS
At line 204964 ENDATA
Problem MODEL has 36960 rows, 43680 columns and 97436 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve determined that the problem was infeasible with tolerance of 1e-08
Analysis indicates model infeasible or unbounded
1 infeasibilities
Analysis indicates model infeasible or unbounded
Perturbing problem by 0.001% of 1 - largest nonzero change 2.0989854e-05 ( 0.0020989854%) - 

In [86]:
import matplotlib.dates as mdates
import datetime

# Beispiel: Erstelle eine Liste von Datumsangaben
start_date = datetime.datetime(2025, 1, 1)
dates = [start_date + datetime.timedelta(hours=i) for i in range(T)]

plt.figure(figsize=(15, 7))
plt.plot(dates, heat_pump_power, label='Wärmepumpenleistung (kW)', color='blue')
plt.xlabel('Datum und Uhrzeit')
plt.ylabel('Leistung (kW)')
plt.title('Verlauf des Wärmepumpenbetriebs über die Zeit')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.show()



AttributeError: module 'matplotlib' has no attribute 'figure'

In [84]:
#Mini-Exkurs Lineare Algebra / Veranschaulichung des Problems 
print("Modelldimensionen: " + str(len(model.constraints)) + " x "+ str(len(model.variables())))
print("------------------")
print("Anzahl der Constraints (m):", len(model.constraints))
print("Anzahl der Variablen (n):", len(model.variables()))


Modelldimensionen: 36960 x 43680
------------------
Anzahl der Constraints (m): 36960
Anzahl der Variablen (n): 43680


# Matrixdarstellung

In der linearen Programmierung gilt folgendes: Constraint (Gleichung) gibt es eine Zeile in der Matrix \( A \), und für jede Variable gibt es eine Spalte. Die Werte in der Matrix \( A \) geben die Koeffizienten der Variablen in den Constraints an.

Die Matrix \( A \) mit Dimension \( 96 \times 96 \):

\[
A =
\begin{bmatrix}
a_{1,1} & a_{1,2} & a_{1,3} & \cdots & a_{1,M} \\
a_{2,1} & a_{2,2} & a_{2,3} & \cdots & a_{2,M} \\
a_{3,1} & a_{3,2} & a_{3,3} & \cdots & a_{3,M} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
a_{N,1} & a_{N,2} & a_{N,3} & \cdots & a_{N,M} \\
\end{bmatrix}

\]



In [81]:
#Amortisationsdauer der Wärmepumpe alleine
tag_kosten_pumpe_only = sum(d * preis for preis in strompreise)/4

tägliche_ersparnisse = jahresrechnung_heizkosten_vor_pumpe/365 - tag_kosten_pumpe_only

print("Average cost for heating before: ", jahresrechnung_heizkosten_vor_pumpe/365)
print("Cost with WP: ",tag_kosten_pumpe_only)
print("Ersparnisse pro Tag: ", tägliche_ersparnisse)
print("Average savings in %: ", tägliche_ersparnisse/(jahresrechnung_heizkosten_vor_pumpe/365)*100)
print("Amortisationsdauer in Jahren: ", anschaffunskosten_pumpe/tägliche_ersparnisse/365)

NameError: name 'strompreise' is not defined

In [82]:
#Amortisationsdauer des Wärmespeichers

# Optimalen Wert der Zielfunktion in einer Variablen speichern
optimal_value = value(model.objective)/4


print("Daily cost with pufferspeicher", optimal_value)
print("Daily cost without pufferspeicher", tag_kosten_pumpe_only)

ersparnisse = tag_kosten_pumpe_only - optimal_value
tage_bis_gelohnt = anschaffungskosten_speicher/ersparnisse

print("Gespart pro Tag in Euro: ", ersparnisse)
print("Amortisationsdauer basierend auf den letzten 24h in Jahren: ", tage_bis_gelohnt/365)

Daily cost with pufferspeicher 8152.534776614993


NameError: name 'tag_kosten_pumpe_only' is not defined

In [83]:
#reale Schätzungen für Amortisationsdauer
#hier reale schätzung, durch vorfaktor basierend auf jahreszeit (da nur 1d wetter daten vorhanden)